# DataUp

Proyecto del curso "Taller de Proyecto". Análisis de entrevistas utilizando procesamiento de voz y técnicas de análisis de texto

In [7]:
#Imports
#Genericas.
import numpy as np
import pandas as pd
from os import path
import matplotlib.pyplot as plt
#Reconocimiento de voz.
import speech_recognition as sr
import pyaudio
#Procesamiento de texto.
import re
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

from nltk.stem.wordnet import WordNetLemmatizer

## 1.- Procesamiento de Voz

In [14]:
#Captura 
def capturaAudio(tiempoEspera = 5):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        #Ajuste a piso de ruido.
        r.adjust_for_ambient_noise(source)
        print('Comience el audio')
        #Iniciar escucha.
        audio = r.listen(source, timeout = tiempoEspera)
        try:
            # for testing purposes, we're just using the default API key
            # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
            # instead of `r.recognize_google(audio)`
            texto = r.recognize_google(audio,language="es-CL")
            print("Google Speech Recognition thinks you said " + texto)
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
        print ('Termino de audio')
        
        return texto

In [20]:
audioPrueba = capturaAudio()

Comience el audio
Google Speech Recognition thinks you said dulzura dulce dulce dulcísimo dulce dulce dulce dulce amargo amargo amargo amargo amargo
Termino de audio


In [21]:
audioPrueba

'dulzura dulce dulce dulcísimo dulce dulce dulce dulce amargo amargo amargo amargo amargo'

## 2.- Análisis del Texto

### 2.1.- Pre-procesamiento

In [22]:
texto = "Un edificio gris, achaparrado, de sólo treinta y 4 plantas. Encima de la entrada principal las palabras: Centro de Incubacion y Condicionamiento de la Central de Londres, y, en un escudo, la divisa del Estado Mundial: Comunidad, Identidad, Estabilidad. Dentro, la enorme sala de la planta baja se hallaba orientada hacia el Norte. Fría a pesar del verano que reinaba en el exterior y del calor tropical de la sala, una luz cruda se filtraba a través de las ventanas buscando ávidamente algún cuerpo amortajado o alguna figura pálida, producto de las disecciones académicas; sin encontrar más que el cristal, el níquel y la brillante porcelana de un laboratorio. Lo invernal respondía a lo invernal. Las batas de los trabajadores eran blancas, y éstos llevaban las manos enfundadas en guantes de goma de un color pálido, como de cadáver. La luz era helada, muerta, fantasmal. Sólo de los amarillos cilindros de los microscopios se lograba arrancar cierta calidad de vida, deslizándose a lo largo de los tubos y formando una dilatada procesión de trazos luminosos que seguían la larga perspectiva de las mesas de trabajo."
pandasText = pd.DataFrame([texto],columns = ['Texto'], index = [1])
texto1 = "Dulzor, 5; Amargor, 1; Burbujas; 10"
texto2 = "El encuestado opino que la bebida tenia un dulzor 5; asigno a amargor un 1 y a burbujas un 10"
pandasText1 = pd.DataFrame([texto1],columns = ['Texto'], index = [1])

#### 2.1.1.- Eliminacion de stopwords y ruido

Removing stopwords: Stop words include the large number of prepositions, pronouns, conjunctions etc in sentences. These words need to be removed before we analyse the text, so that the frequently used words are mainly the words relevant to the context and not common words used in the text.

In [23]:
#Lista de stop words en espanol. Util para elminaras del texto original.
stopWords = set(stopwords.words("spanish"))

In [37]:
def pasoPanda(texto):
    output = pd.DataFrame([texto],columns = ['Texto'], index = [1])
    return output

def preProcesamiento(textoPandas,stopWords, deleteStopWords = 1, lemanizar= 0):
    #Funcion que realiza el preProcesamiento del texto.
    
    #Convertir a Pandas.
    text = textoPandas['Texto'][1]
    #Pasar a minusculas.
    text = text.lower()
    #Eliminar comas y simbolos.
    text = re.sub(r'[^\w\s]','',text)
    #Convertir a vector de palabras.
    text = text.split()
    #Eliminar stopWords
    text = [word for word in text if not word in stopWords]
    #Decidir si lemanizar o no
    if lemanizar == 1:
        lem = WordNetLemmatizer()
        text = np.array([lem.lemmatize(word) for word in text])
    elif lemanizar == 2:
        stem = PorterStemmer()
        text = np.array([stem.stem(word) for word in text])
    return text

In [25]:
#Generar el texto preProcesado. Se entrega un vector con las palabras clave.
text = preProcesamiento(pandasText,stopWords,lemanizar=1)

In [26]:
text

array(['edificio', 'gris', 'achaparrado', 'sólo', 'treinta', '4',
       'plantas', 'encima', 'entrada', 'principal', 'palabras', 'centro',
       'incubacion', 'condicionamiento', 'central', 'londres', 'escudo',
       'divisa', 'mundial', 'comunidad', 'identidad', 'estabilidad',
       'dentro', 'enorme', 'sala', 'planta', 'baja', 'hallaba',
       'orientada', 'hacia', 'norte', 'fría', 'pesar', 'verano',
       'reinaba', 'exterior', 'calor', 'tropical', 'sala', 'luz', 'cruda',
       'filtraba', 'través', 'ventanas', 'buscando', 'ávidamente',
       'algún', 'cuerpo', 'amortajado', 'alguna', 'figura', 'pálida',
       'producto', 'disecciones', 'académicas', 'encontrar', 'cristal',
       'níquel', 'brillante', 'porcelana', 'laboratorio', 'invernal',
       'respondía', 'invernal', 'bata', 'trabajadores', 'blancas',
       'éstos', 'llevaban', 'manos', 'enfundadas', 'guantes', 'goma',
       'color', 'pálido', 'cadáver', 'luz', 'helada', 'muerta',
       'fantasmal', 'sólo', 'amari

In [27]:
def stringNumerico (word):
    #Funcion que verifica si un string es numerico
    try:
        val = int(word)
        output = True
    except ValueError:
        output = False
    return output

#Funcion que realiza la busqueda de la calificacion.
def buscarValorAsociado(caracteristica, vectorTexto):
    indiceCaracteristica = np.where(vectorTexto == caracteristica)
    calificacion = int(vectorTexto[indiceCaracteristica[0]+1])
    return calificacion

In [28]:
#Generar el texto preProcesado. Se entrega un vector con las palabras clave.
text1 = preProcesamiento(pandasText1,stopWords,lemanizar=1)

In [29]:
text1

array(['dulzor', '5', 'amargor', '1', 'burbujas', '10'], dtype='<U8')

In [30]:
buscarValorAsociado('amargor',text1)

1

In [38]:
text2 = pasoPanda(audioPrueba)
text2 = preProcesamiento(text2,stopWords,lemanizar=2)

In [39]:
text2

array(['dulzura', 'dulc', 'dulc', 'dulcísimo', 'dulc', 'dulc', 'dulc',
       'dulc', 'amargo', 'amargo', 'amargo', 'amargo', 'amargo'],
      dtype='<U9')